# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [1]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [3]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [4]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [62]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

12246825189350298909796

## Use this to import default tbears wallet and run the transactions


In [14]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb


786640639999000000000000000

In [14]:
contracts_file_custom = "shubh_yeouido_contracts_20210112084238.pkl"

In [15]:
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# Deploying all contracts


In [11]:


# deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'liquidationManager']
# for directory in {"score"}:
deploy= ['snapshot']
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "omm_token":
        params = {'_initialSupply':0, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x9f9609999102460a6461cbb38abbed150d390ee43cf0a58dbdff6f39986a3667', 'blockHeight': 12772046, 'blockHash': '0xda6706230504889adca48e15144818d0ac317987e4a9655407f891201da2dcd3', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxa3bd48427581e6a41e9f03ee4bea14d5e506705b', 'stepUsed': 1052762400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1052762400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [45]:
#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

# now = datetime.utcnow()

# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# filename=network+'_'+filename
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x359ae4db5fe8250861bf98ea41cc0f3576556e81c10941833b8fcd23a0252c08', 'blockHeight': 12135480, 'blockHash': '0x87feb49e50a77ee562a32b6a4b5fd5043162b91fe21014fd418dfcf3bc55b6b5', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx30541a191a4261ef375931113637b8ce5ca227b0', 'stepUsed': 1143737240, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1143737240, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cxa6ababb9a42d12d3e5037076a3274fd0aca94380',
 'feeProvider': 'cx5e02eb90c899a02a4a54738e8d6e10955349a9f5',
 'lendingPool': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
 'lendingPoolCore': 'cxef84c3aec46054d31803c03b2011eb8ed4efaabc',
 'lendingPoolDataProvider': 'cx1a7475863d1d2d5971392fc0dadc554c44713ec5',
 'oToken': 'cxbdea69d5d0dab660bc6f0572db03e98f466dd897',
 'priceOracle': 'cx54dfebc01aace3e0fbbb58442b361368689d0cde',
 'sample_token': 'cx7e2c54238f86a6ebc65aecae9e9203a7210a89eb',
 'liquidationManager': 'cx0e60be7ad2a920ede13a5b4db10e7f7d1b09494d',
 'worker_token': 'cxcc293a60b4abb77e2ef9c41cf4035b5368898f08',
 'snapshot': 'cx29d7814128d55747b38e6e3ffd23695f4a513906',
 'rewards': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
 'oICX': 'cx30541a191a4261ef375931113637b8ce5ca227b0'}

## Updating contracts

In [130]:
contract_ = 'rewards'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update rewards cxd100e81ac0839b6130bf4c6806489a9af6274db6


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x7eaeea2462e1eb0a3188bc30e9f1848a8349e8951e0b29a966720ed0f0bbe154',
 'blockHeight': 12705020,
 'blockHash': '0x43785ae8a02adbcd0d010588440d499704782d0d45c2f182b4f8b21a2e07a3a0',
 'txIndex': 1,
 'to': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
 'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
 'stepUsed': 1716815200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1716815200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [10]:
params = {'_initialSupply':500000000, '_decimals': 18}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('sicx'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['sicx'] = res.get('scoreAddress', '')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x7fae3a2fcabdb34636b054ec79bceec0b4252af13941af9285ec1de8915130fa', 'blockHeight': 3452394, 'blockHash': '0xd111e45227c8406c5e5b01e6f55a68c4be231d3993a5fcc5e92531fa463e2eb6', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxd67aead6c00c8ccf7c2711024264c1c9d13ac85b', 'stepUsed': 1053603200, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1053603200, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [43]:
contracts['sicx']="cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281"
contracts['staking']="cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c"
contracts['oicx'] ="cxf6e3d64db674c0edbf732462e80e2e3352280548""


In [16]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'delegation': 'cx5ecc21ade5e4a90586bde6451da3b0d733dd11d6',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'governance': 'cx9e0ecfd9cc9a0c1cee8fc4a4701e970554835f54',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'omm_token': 'cxff036606b11066a142262431b089e8d0af145bef',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'rewards': 'cxcccee742d4c1b7a3a84a20a41691d302e25ab3a0',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'snapshot': 

## Saving the pkl file


In [13]:
now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename="shubh_yeouido_contracts_20210112084238.pkl"
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

saving contracts list as shubh_yeouido_contracts_20210112084238.pkl


# Configuring SCOREs's parameters

In [11]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress','params': {'_address':contracts['liquidationManager']}},
            {'contract': 'lendingPool', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'lendingPool', 'method': 'setOICXAddress', 'params':{'_address': contracts['oICX']}},
            {'contract': 'lendingPool', 'method': 'setRewardAddress','params': {'_address':contracts['rewards']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"ICX"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
            {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},  
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
            {'contract': 'lendingPoolCore', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
            {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'liquidationManager', 'method': 'setOracleAddress','params': {'_address':contracts['priceOracle']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},
            {'contract': 'rewards', 'method': 'setLendingPool','params': {'_val':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapsot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},]

In [23]:
settings=[  {'contract': 'rewards', 'method': 'setDistPercentage','params': {'_deposit':25*10**16,'_borrow':25*10**16,'_ommUSDb':10**17,''}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}
            ]

## Rewards settings

In [ ]:
settings=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_val':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}
            ]

In [13]:
local_settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**16}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
              {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
                {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress',
'params': {'_address':contracts['liquidationManager']}},
                {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
                {'contract': 'liquidationManager', 'method': 'setOracleAddress',
'params': {'_address':contracts['priceOracle']}},
                {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress',
'params': {'_address':contracts['feeProvider']}},]

In [24]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_val': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788'} on the rewards contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolCore, with parameters {'_val': 'cxef84c3aec46054d31803c03b2011eb8ed4efaabc'} on the rewards contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSnapshot, with parameters {'_val': 'cx29d7814128d55747b38e6e3ffd23695f4a513906'} on the rewards contract.


ConnectTimeout: HTTPSConnectionPool(host='bicon.net.solidwallet.io', port=443): Max retries exceeded with url: /api/v3 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7f377ba786d0>, 'Connection to bicon.net.solidwallet.io timed out. (connect timeout=10)'))

## Adding USDb reserve in LendingPoolCore



In [14]:
params ={"_reserve": {"reserveAddress":contracts['sample_token'],"oTokenAddress":contracts['oToken'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb776ef073179422778019c8d5e21cd416f0ca055a85b0037837224726c2d0c77',
 'blockHeight': 12303952,
 'blockHash': '0xe2add83ce977846e3058a77420b1a87786860a87dd2b28849a4bd72edb6a8e49',
 'txIndex': 1,
 'to': 'cxef84c3aec46054d31803c03b2011eb8ed4efaabc',
 'stepUsed': 277040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 277040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [15]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xdb694fb9b91b8a471ea4005e57d509e755a4119b2673d58efec1e7450f80d65c',
 'blockHeight': 12303956,
 'blockHash': '0x575c4814bd86d7c868cfb0f2e0807eb903f3df5daec96d09f5b7c6719274fc7f',
 'txIndex': 1,
 'to': 'cxef84c3aec46054d31803c03b2011eb8ed4efaabc',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Transfering USDb to wallets

In [10]:
private2="b3d4e18d733f05dc21f326ba4fe150514874327c738c4a19ddd357d0e8e2efed"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="c3d5e9afe753a07c983bb14e34f73907e40a997793ce2f7d2895cddec27f81c6"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

private4="2ffb9ef51826d555189d4804161058036cf87ae8290f1a308a79d045f6cdd428"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())
wallets=[deployer_wallet2,deployer_wallet3,deployer_wallet4]

hxd29037f1eb8c3f0e8630d797edc7410507c0089a
hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
hx65902dd4139d04765534a4539319be874ad7aa0b


In [17]:
for wallet in wallets:
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18} 
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sample_token'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


In [15]:
params = {'_owner': "hxd29037f1eb8c3f0e8630d797edc7410507c0089a"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)


print(int(response,0))

0


In [16]:
for wallet in wallets:
    print(wallet.get_address())
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18}
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sicx'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(get_tx_result(tx_hash))

hxd29037f1eb8c3f0e8630d797edc7410507c0089a


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x08dcb6a468c9830ae13cb6abe2a18c035def9df8f839a35fa968a184d5defbc2', 'blockHeight': 12399911, 'blockHash': '0xc30a1d3a584c8558861c51221f35774dee7a84efa5dccb75286ed159606d32c6', 'txIndex': 1, 'to': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', 'stepUsed': 148400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xe4478ad3aaa74fc5f173986e6282a57fbb0c46136798f7c416ef9cd4217bf08a', 'blockHeight': 12399913, 'blockHash': '0x8de2a4100afeb72b670688ce580d41214579a32515dff4c32de69d30cafd86cf', 'txIndex': 1, 'to': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', 'stepUsed': 148400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xb1c7ef8bc1fbc9afdac0e16362904d2af9c459051435d9248217b2731647201a', 'blockHeight': 12399916, 'blockHash': '0xc2726f37cb75f2ac3b590dca652d6b0a367935b52d295e3cf05015ecfc13ab92', 'txIndex': 1, 'to': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', 'stepUsed': 148400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Transfering icx
 (the below cells are optional,if you want to tranfer icx to any of the wallet than only you need to run the cells)


In [17]:
password = "test1_Account"
icx_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
icx_wallet.get_address()
icon_service.get_balance(icx_wallet.get_address())

Raised KeyStoreException while loading the wallet by the keystore file because the file is not found.
Traceback (most recent call last):
  File "/home/kailash/.local/lib/python3.8/site-packages/iconsdk/wallet/wallet.py", line 101, in load
    with open(file_path, 'rb') as file:
FileNotFoundError: [Errno 2] No such file or directory: './keystores/keystore_test1.json'


KeyStoreException: File is not found. (Key store error)

# run the below cell only when you want to transfer ICX to a wallet
change the wallet address and the amount of icx to be transferred to that wallet

In [18]:
# hxd29037f1eb8c3f0e8630d797edc7410507c0089a
# hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
# hx65902dd4139d04765534a4539319be874ad7aa0b
wallet = "hxd29037f1eb8c3f0e8630d797edc7410507c0089a" 
amount = 10000*10**18                               #change the amount of icx to transfer here
transaction = TransactionBuilder()\
    .from_(icx_wallet.get_address())\
    .to(wallet)\
    .nid(3) \
    .nonce(100) \
    .value(amount)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, icx_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

NameError: name 'icx_wallet' is not defined

In [69]:
contract_ = 'rewards'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update rewards cxd100e81ac0839b6130bf4c6806489a9af6274db6


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x704e6da25a77f0c6e72f29ea143ac6cf311cfebd3134a99116c17331ad5c1369',
 'blockHeight': 12742099,
 'blockHash': '0xe4b1e6a05ae96b14300205c7efd8b641bd16b8eec729a474453e8104d5c2e042',
 'txIndex': 1,
 'to': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
 'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
 'stepUsed': 1724354720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1724354720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [68]:
print(deployer_wallet.get_address())
params = {"_amount":   10**18 }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(10**18)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("deposit")\
    .params(params)\
    .build()

signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

hx91bf040426f226b3bfcd2f0b5967bbb0320525ce


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xd07ad600ddb345272aa7361f23199c5e9e8b7149ef5bf9d70ec71df12d8c3578',
 'blockHeight': 12742002,
 'blockHash': '0x7d262b9aa9e4afa73d7281d8876bdbe9ca206c7f99bfcfbd986c301337bbb492',
 'txIndex': 1,
 'to': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
 'stepUsed': 1034860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1034860,
 'eventLogs': [{'scoreAddress': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
    'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
    '0xde0b6b3a7640000'],
   'data': []},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
    'cxef84c3aec46054d31803c03b2011eb8ed4efaabc',
    '0xdb2106f50518fd6'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Mint(Address,int,bytes)',
    'cxef

In [70]:
params = {"_amount": 10**18 ,"_reserve":contracts['sicx'] }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("borrow")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x76ab25fd5ebbcc4cf853a7e9dd6f7c146c0fe582da07e0de71b8a497095d5046',
 'blockHeight': 12742104,
 'blockHash': '0x9e8a2916a7509d73eb45f1fce0a0d07d67053f2ef8aa11fe7e67617043624774',
 'txIndex': 1,
 'to': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
 'stepUsed': 2306180,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2306180,
 'eventLogs': [{'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'worker',
    'hxd64a458c9714775e695f63b2d86f66a22c23f85a',
    '0x2a5a058fc295ed000000'],
   'data': []},
  {'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'daoFund',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x152d02c7e14af6800000'],
   'data': []},
  {'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'ommICX',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
    '0x152d02c7e14af6

In [13]:
contracts['sicx']

'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'